# Standardised Approach for Counterparty Credit Risk

Abbreviations: <br >
- $VM$: Variation Margin
- $EAD$: Exposure At Default under SA
- $RC$: replacement cost (different for margined and unmargined)
- $PFE$: Potential Future Exposure
- $V$: Current Market Value of the Derivative in the netting set
- $C$: Net harcut value of the net collateral held
- $ICA$: Independent Collateral Amount
- $TH$: Positive threshold before the counterparty must send the bank collateral
- $MTA$: Minimum transfer amount
- $NICA$: Net independent Collateral Amount
- $MF_i^{type}$: Maturity Factor
- $M_i$: Maturity (i remaining maturity floored by 10 business days)
- $S_i$: Start
- $E_i$: end date
- $T_i$: Contractual Exercise Date
- ${SD}_i$: Supervisory Duration
- $\delta_i$: Supervisory delta Adjustments
- $MPOR_i$: Margin period of risk appropriate for the margin agreement containing the transaction i

Formulas: <br> 
- EAD = alpha * (RC + PFE) 
- RC (Margined) = max{V - C;TH + MTA - NICA; 0}
- PFE = multiplier * $AddOn^{aggregate}$
- multipiler = min{1; Floor + (1 - Floor) * exp ($\frac{V - C}{2* (1-Floor) * AddOn^{aggregate} }$)}
- $AddOn^{aggregate}$ = $\sum_{a} AddOn^a$
- ${SD}_i$ = $\frac {exp(-0.05*S_i)-exp(-0.05*Ei)} {0.05}$
- ${MF}_i^{unmargined}$ = $\sqrt{\frac {min\{M_i; 1 year\}}{1year}}$
- ${MF}_i^{(margined)}$ = $\frac {3} {2} \sqrt{\frac{MPOR_i}{1year}}$

<a href="https://www.bis.org/publ/bcbs279.pdf">PDF Reference</a>

In [1]:
# csv file sample input


<h3>Example 1: <h3/>

In [2]:
import pandas as pd
import math
import numpy as np

In [3]:
fileName = "example_1_input.csv"
example_1_df = pd.read_csv(fileName)

In [4]:
example_1_df

,Trade #,Nature,Residual maturity,Base currency,Notional (thousands),Pay Leg (*),Receive Leg (*),Market value (thousands)
0,1,Interest rate swap,10 years,USD,10000,Fixed,Floating,30
1,2,Interest rate swap,4 years,USD,10000,Floating,Fixed,-20
2,3,European swaption,1 into 10 years,EUR,5000,Floating,Fixed,50


rc(df) takes in a dataframe and compute the RC of a given file

In [5]:
def rc(df):
    v_c = example_1_df[['Market value (thousands)']].sum()
    r_c = max(int(v_c), 0)
    return r_c

In [6]:
rc(example_1_df)

60

In [7]:
example_1_df.dtypes

Trade #                      int64
Nature                      object
Residual maturity           object
Base currency               object
Notional (thousands)         int64
Pay Leg (*)                 object
Receive Leg (*)             object
Market value (thousands)     int64
dtype: object

In [8]:
# Creating a new dataframe for further analysis
data = {}
hedging_set = example_1_df['Base currency'].tolist()
data.update({"Hedging set": hedging_set})
data

{'Hedging set': ['USD', 'USD', 'EUR']}

In [9]:
# convert residual maturity to two lists, S and E by taking in
# three variables, emptyEi, emptySi and residual maturity
def residual_maturity_converter(Ei, Si, residual_maturity):
    for item in residual_maturity:
        item = str(item)
        idx = item.find("into")
        if (idx == -1): # S would be 0
            Si.append(0)
            end = int(item[:item.find("year") - 1])
            Ei.append(end)
        else:
            idx -= 1
            start = int(item[:idx])
            Si.append(start)
            idx += 5
            end = int(item[idx:item.find("year") - 1])
            Ei.append(end)
            
        for i in range(0,len(Ei)):
            Ei[i] = Ei[i] + Si[i]
        

In [10]:
#SD_generator takes in Si and Ei and generate the appropriate SD as a list
def SD_generator(Si, Ei, SD):
    if (len(Si) != len(Ei)):
        return "Error"
    for i in range(0, len(Si)):
        si = Si[i]
        ei = Ei[i]
        sd = (math.exp(-0.05*si) - math.exp(-0.05*ei)) / 0.05
        SD.append(sd)

In [11]:
Ei = []
Si = []
residual_maturity_converter(Ei, Si, example_1_df['Residual maturity'])

In [12]:
Ei

[10, 4, 11]

In [13]:
Si

[0, 0, 1]

In [14]:
data.update({"Notional (thousands)": example_1_df['Notional (thousands)'].tolist(),
            "Si": Si, "Ei":Ei})
data

{'Hedging set': ['USD', 'USD', 'EUR'],
 'Notional (thousands)': [10000, 10000, 5000],
 'Si': [0, 0, 1],
 'Ei': [10, 4, 11]}

In [15]:
SD = []
SD_generator(Si, Ei, SD)
SD

[7.8693868057473315, 3.6253849384403636, 7.485592282404547]

In [17]:
Adj_notion = np.array(SD) * np.array(example_1_df['Notional (thousands)'].tolist())

In [20]:
data.update({"SDi": SD,
            "Adjusted notional (thousands)": Adj_notion.tolist()})
data

{'Hedging set': ['USD', 'USD', 'EUR'],
 'Notional (thousands)': [10000, 10000, 5000],
 'Si': [0, 0, 1],
 'Ei': [10, 4, 11],
 'SDi': [7.8693868057473315, 3.6253849384403636, 7.485592282404547],
 'Adjusted notional (thousands)': [78693.86805747332,
  36253.849384403635,
  37427.961412022734]}

In [22]:
middle_df = pd.DataFrame(data)
middle_df

,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands)
0,USD,10000,0,10,7.869387,78693.868057
1,USD,10000,0,4,3.625385,36253.849384
2,EUR,5000,1,11,7.485592,37427.961412
